## To relate isolation (sol) to the Urban Heat Island (UHI). Analysis of the isolation variable.

In [1]:
import matplotlib
matplotlib.use("Agg")  # backend no interactivo
import matplotlib.pyplot as plt
import os
import pandas as pd



In [2]:
import os
import pandas as pd


# Absolute path
RAW_AEMET_DIR = os.path.abspath("../data/raw/aemet")

# Loading data CSVs
dfs = []
files = [f for f in os.listdir(RAW_AEMET_DIR) if f.endswith(".csv")]

for fname in files:
    df = pd.read_csv(os.path.join(RAW_AEMET_DIR, fname))
    df["archivo"] = fname
    dfs.append(df)

meteo = pd.concat(dfs, ignore_index=True)

# cleaning and preprocessing
meteo["fecha"] = pd.to_datetime(meteo["fecha"], errors="coerce")

num_cols = ["tmed","tmax","tmin","prec","sol","hrMedia","presMax","presMin","altitud"]
for col in num_cols:
    if col in meteo.columns:
        meteo[col] = pd.to_numeric(meteo[col], errors="coerce")

meteo = meteo.dropna(subset=["fecha"])

meteo["year"] = meteo["fecha"].dt.year


In [3]:
# Valid years with at least 250 days of data
valid_years = (
    meteo.groupby(["nombre", "year"])["sol"]
    .count()
    .reset_index()
    .rename(columns={"sol": "days"})
)

valid_years = valid_years[valid_years["days"] >= 170]

meteo_clean = meteo.merge(valid_years[["nombre", "year"]], on=["nombre", "year"])


In [4]:
# Count days with sunshine in METEO (raw)
counts_raw = (
    meteo.groupby(["nombre", "year"])["sol"]
    .apply(lambda x: x.notna().sum())
    .reset_index()  # sin "name"
    .rename(columns={"sol": "n_days_with_sol_raw"})
)

# Count days with sunshine in METEO_CLEAN (if exists)
if "meteo_clean" in globals():
    counts_clean = (
        meteo_clean.groupby(["nombre", "year"])["sol"]
        .apply(lambda x: x.notna().sum())
        .reset_index()
        .rename(columns={"sol": "n_days_with_sol_clean"})
    )
else:
    counts_clean = pd.DataFrame(columns=["nombre", "year", "n_days_with_sol_clean"])

# Resume per station - raw
summary_raw = (
    counts_raw.groupby("nombre")["n_days_with_sol_raw"]
    .agg(["count", "min", "max", "mean"])
    .reset_index()
    .sort_values("count", ascending=False)
)

summary_raw.head(50)


,nombre,count,min,max,mean
0,BARCELONA AEROPUERTO,46,230,366,348.434783
2,"BARCELONA, FABRA",46,224,366,360.413043
5,MONTSERRAT,46,0,366,163.347826
6,SABADELL AEROPUERTO,37,0,0,0.000000
4,MANRESA,21,0,366,231.047619
7,VILAFRANCA DEL PENEDÈS,18,0,0,0.000000
1,"BARCELONA, DRASSANES",17,0,0,0.000000
3,IGUALADA,13,0,0,0.000000


In [5]:
# List of station with no sunshine days recorded
no_sol_stations = counts_raw.groupby("nombre")["n_days_with_sol_raw"].sum().reset_index()
no_sol_stations = no_sol_stations[no_sol_stations["n_days_with_sol_raw"] == 0]
no_sol_stations


,nombre,n_days_with_sol_raw
1,"BARCELONA, DRASSANES",0
3,IGUALADA,0
6,SABADELL AEROPUERTO,0
7,VILAFRANCA DEL PENEDÈS,0


In [6]:
# Mininum days with sunshine to consider a year valid
MIN_DAYS_SOL = 200  

# Count days with sunshine and mean sunshine per year
sol_counts = (
    meteo.groupby(["nombre","year"], as_index=False)["sol"]
    .agg(n_days_with_sol = lambda x: x.notna().sum(), sol_mean = lambda x: x.dropna().mean())
)

# Filter years with enough days with sunshine
sol_valid = sol_counts[sol_counts["n_days_with_sol"] >= MIN_DAYS_SOL].copy()
sol_valid = sol_valid.rename(columns={"sol_mean":"sol"})

# Create annual_sol DataFrame
annual_sol = sol_valid[["nombre","year","sol","n_days_with_sol"]].copy()

# Information about valid station-year pairs
print("Total station-year pairs with >= {} days: {}".format(MIN_DAYS_SOL, len(annual_sol)))
display(annual_sol.groupby("nombre")["year"].count().reset_index().rename(columns={"year":"n_valid_years"}).sort_values("n_valid_years", ascending=False))


Total station-year pairs with >= 200 days: 126


,nombre,n_valid_years
0,BARCELONA AEROPUERTO,46
1,"BARCELONA, FABRA",46
3,MONTSERRAT,21
2,MANRESA,13


In [7]:
def plot_sol_trend_from_annual_sol(annual_sol_df, station_name, min_years=3, outdir="../reports/sol"):
    # Filtrar datos de la estación
    data = annual_sol_df[annual_sol_df["nombre"] == station_name].copy()

    # Aseguramos que 'sol' es numérico
    data["sol"] = pd.to_numeric(data["sol"], errors="coerce")
    data = data.dropna(subset=["sol"]).sort_values("year")

    if data.empty:
        print(f"⚠️ No annual_sol data for '{station_name}'.")
        return None
    if len(data) < min_years:
        print(f"⚠️ Only {len(data)} valid years for '{station_name}' (need >= {min_years}).")
        return None

    # Pasar a listas de Python (sin NumPy)
    pairs = []
    for _, row in data.iterrows():
        y = row["year"]
        s = row["sol"]
        if pd.notna(y) and pd.notna(s):
            try:
                pairs.append((float(y), float(s)))
            except ValueError:
                continue

    if len(pairs) < min_years:
        print(f"⚠️ After cleaning, only {len(pairs)} valid years for '{station_name}'.")
        return None

    years, sol_vals = zip(*pairs)
    years = list(years)
    sol_vals = list(sol_vals)

    # 🔥 Regresión lineal en Python puro (sin NumPy)
    n = len(years)
    sum_x = sum(years)
    sum_y = sum(sol_vals)
    sum_x2 = sum(x*x for x in years)
    sum_xy = sum(x*y for x, y in zip(years, sol_vals))

    denom = n * sum_x2 - sum_x**2
    if denom == 0:
        print(f"⚠️ Cannot compute trend for '{station_name}' (denominator = 0).")
        return None

    slope = (n * sum_xy - sum_x * sum_y) / denom
    intercept = (sum_y - slope * sum_x) / n
    trend = [slope * x + intercept for x in years]

    # Plot
    plt.figure(figsize=(8,4))
    plt.plot(years, sol_vals, marker='o', lw=1, label="Mean annual solar hours")
    plt.plot(years, trend, '--', label=f"Trend: {slope*10:.1f} h/decade")
    plt.title(f"Annual Insolation — {station_name}")
    plt.xlabel("Year")
    plt.ylabel("Hours of sunshine")
    plt.grid(True, alpha=0.3)
    plt.legend()

    # Guardar
    outdir_abs = os.path.abspath(outdir)
    os.makedirs(outdir_abs, exist_ok=True)

    safe_station = (
        station_name.lower()
        .replace(" ", "_")
        .replace(",", "")
        .replace("·", "")
        .replace("'", "")
    )
    sol_filename = f"annual_sol_trend_{safe_station}.png"
    outpath = os.path.join(outdir_abs, sol_filename)

    plt.savefig(outpath, dpi=300, bbox_inches="tight")
    plt.close()  # cerrar figura

    print(f"📈 Tendència {station_name}: {slope:.3f} h/any ({slope*10:.1f} h/dècada) — saved: {outpath}")
    return {"station": station_name, "slope_h_per_year": slope, "n_years": n}


In [8]:
stations_try = [
    "BARCELONA, DRASSANES",
    "BARCELONA, FABRA",
    "BARCELONA AEROPUERTO",
    "SABADELL AEROPUERTO",
    "MONTSERRAT"
]

results = []
for st in stations_try:
    print("\n>>> Probando:", st)
    r = plot_sol_trend_from_annual_sol(annual_sol, st)
    if r is not None:
        results.append(r)

pd.DataFrame(results)



>>> Probando: BARCELONA, DRASSANES
⚠️ No annual_sol data for 'BARCELONA, DRASSANES'.

>>> Probando: BARCELONA, FABRA
📈 Tendència BARCELONA, FABRA: 0.040 h/any (0.4 h/dècada) — saved: d:\PersonalProjects\TFG\TFG-UHI Barcelona\reports\sol\annual_sol_trend_barcelona_fabra.png

>>> Probando: BARCELONA AEROPUERTO
📈 Tendència BARCELONA AEROPUERTO: -0.007 h/any (-0.1 h/dècada) — saved: d:\PersonalProjects\TFG\TFG-UHI Barcelona\reports\sol\annual_sol_trend_barcelona_aeropuerto.png

>>> Probando: SABADELL AEROPUERTO
⚠️ No annual_sol data for 'SABADELL AEROPUERTO'.

>>> Probando: MONTSERRAT
📈 Tendència MONTSERRAT: -0.029 h/any (-0.3 h/dècada) — saved: d:\PersonalProjects\TFG\TFG-UHI Barcelona\reports\sol\annual_sol_trend_montserrat.png


,station,slope_h_per_year,n_years
0,"BARCELONA, FABRA",0.040027,46
1,BARCELONA AEROPUERTO,-0.006561,46
2,MONTSERRAT,-0.029382,21


In [9]:
# Function to count days with sunshine per year, checking sunshine data availability
def count_sol_days(df, station_name):
    df_st = df[df["nombre"] == station_name]
    if df_st.empty:
        print(f"⚠️ No existe la estación '{station_name}' en el dataframe.")
        return None
    
    result = (
        df_st.groupby("year")["sol"]
        .apply(lambda x: x.notna().sum())
        .reset_index()
        .rename(columns={"sol": "n_days_with_sol"})
        .sort_values("year")
    )
    
    print(f"\n📍 Estación: {station_name}")
    print(result)
    print("\nTotal años detectados:", result.shape[0])
    print("Años con ≥ 50 días de sol:", (result["n_days_with_sol"] >= 50).sum())
    return result

# Drassanes and Sabadell airport
count_drassanes = count_sol_days(meteo, "BARCELONA, DRASSANES")
count_sabadell = count_sol_days(meteo, "SABADELL AEROPUERTO")



📍 Estación: BARCELONA, DRASSANES
    year  n_days_with_sol
0   2009                0
1   2010                0
2   2011                0
3   2012                0
4   2013                0
5   2014                0
6   2015                0
7   2016                0
8   2017                0
9   2018                0
10  2019                0
11  2020                0
12  2021                0
13  2022                0
14  2023                0
15  2024                0
16  2025                0

Total años detectados: 17
Años con ≥ 50 días de sol: 0

📍 Estación: SABADELL AEROPUERTO
    year  n_days_with_sol
0   1980                0
1   1981                0
2   1982                0
3   1983                0
4   1984                0
5   1985                0
6   1986                0
7   1987                0
8   1988                0
9   1989                0
10  1990                0
11  1991                0
12  1992                0
13  1993                0
14  1994           

La variable de insolación únicamente está disponible en las estaciones equipadas con heliógrafo (Fabra, Barcelona Aeropuerto y Montserrat). En el caso de las estaciones urbanas automáticas (Barcelona Drassanes) y aeronáuticas (Sabadell Aeropuerto), dicha variable no se mide oficialmente, por lo que el análisis de insolación se limita a las estaciones que disponen de este registro.

In [10]:
# Stations with sunshine data (a urban, an airport and a mountain)
stations_sol = [
    "BARCELONA, FABRA",
    "BARCELONA AEROPUERTO",
    "MONTSERRAT"
]

annual_sol_filtered = annual_sol[annual_sol["nombre"].isin(stations_sol)].copy()

annual_sol_filtered.groupby("nombre")["year"].count()


nombre
BARCELONA AEROPUERTO    46
BARCELONA, FABRA        46
MONTSERRAT              21
Name: year, dtype: int64

In [11]:
# Function to plot sunshine trend
# def plot_sol_trend(df, station_name, outdir="../reports/sol"):
#     data = df[df["nombre"] == station_name].dropna(subset=["sol"]).sort_values("year")
#     if data.empty or len(data) < 3:
#         print(f"⚠️ Not enough insolación data for {station_name}")
#         return
    
#     years = data["year"].values
#     sol_vals = data["sol"].values
    
#     slope, intercept = np.polyfit(years, sol_vals, 1)
#     trend = slope * years + intercept
    
#     plt.figure(figsize=(8,4))
#     plt.plot(years, sol_vals, marker="o", label="Annual sunshine hours")
#     plt.plot(years, trend, "--", label=f"Trend: {slope*10:.1f} h/decade")
    
#     plt.title(f"Annual Insolation — {station_name}")
#     plt.xlabel("Year")
#     plt.ylabel("Sunshine hours")
#     plt.grid(True, alpha=0.3)
#     plt.legend()
    
#     os.makedirs(outdir, exist_ok=True)
#     fname = f"insolation_trend_{station_name.replace(' ', '_').lower()}.png"
#     plt.savefig(os.path.join(outdir, fname), dpi=300, bbox_inches="tight")
    
#     plt.show()
    
#     print(f"📈 {station_name}: slope={slope:.3f} h/year ({slope*10:.1f} h/decade)")


In [12]:
# Trying function
# plot_sol_trend(annual_sol_filtered, "BARCELONA, FABRA")
# plot_sol_trend(annual_sol_filtered, "BARCELONA AEROPUERTO")
# plot_sol_trend(annual_sol_filtered, "MONTSERRAT")


In [13]:
plt.figure(figsize=(12,6))

for st in stations_sol:
    data = annual_sol_filtered[annual_sol_filtered["nombre"] == st]
    plt.plot(data["year"], data["sol"], marker="o", label=st)

plt.title("Insolation Comparison: Urban vs Periurban vs Rural")
plt.xlabel("Year")
plt.ylabel("Sunshine hours")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


C:\Users\roger\AppData\Local\Temp\ipykernel_20268\2357065701.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
# --- A: Annual temps and ATD ---
import pandas as pd

# Asumimos que meteo_clean existe en este notebook (si no, recrearlo como hiciste antes)
# Crear medias anuales de temperaturas
annual_temp = (
    meteo_clean
    .groupby(["nombre", "year"], as_index=False)
    .agg({
        "tmin": "mean",
        "tmax": "mean",
        "tmed": "mean"
    })
)

# ATD = Tmax - Tmin
annual_temp["ATD"] = annual_temp["tmax"] - annual_temp["tmin"]

# quick check
display(annual_temp[annual_temp["nombre"].isin(["BARCELONA, FABRA","BARCELONA AEROPUERTO","MONTSERRAT"])].head())


,nombre,year,tmin,tmax,tmed,ATD
0,BARCELONA AEROPUERTO,1980,10.355738,19.818033,15.086612,9.462295
1,BARCELONA AEROPUERTO,1981,10.925753,20.329315,15.627397,9.403562
2,BARCELONA AEROPUERTO,1982,11.512055,20.306301,15.909315,8.794247
3,BARCELONA AEROPUERTO,1983,11.117260,20.457808,15.787397,9.340548
4,BARCELONA AEROPUERTO,1984,9.955191,19.557377,14.755464,9.602186


In [15]:
# --- B: Rebuild annual_sol robustly (use only years with enough days) ---
MIN_DAYS_SOL = 150  # ajusta si quieres: 200 es más conservador

# Contar dias con sol por estacion-año y calcular media si pasan el umbral
sol_counts = (
    meteo.groupby(["nombre","year"], as_index=False)["sol"]
    .agg(n_days_with_sol = lambda x: x.notna().sum(),
         sol_mean = lambda x: x.dropna().mean())
)

# Filtrar años con suficientes dias
sol_valid = sol_counts[sol_counts["n_days_with_sol"] >= MIN_DAYS_SOL].copy()
sol_valid = sol_valid.rename(columns={"sol_mean":"sol"})

# annual_sol (solo años válidos)
annual_sol = sol_valid[["nombre","year","sol","n_days_with_sol"]].copy()

# Filtrar sólo estaciones que vamos a analizar (Fabra, BCN Airport, Montserrat)
stations_sol = ["BARCELONA, FABRA", "BARCELONA AEROPUERTO", "MONTSERRAT"]
annual_sol_filtered = annual_sol[annual_sol["nombre"].isin(stations_sol)].copy()

print("Years per station with >= {} days:".format(MIN_DAYS_SOL))
display(annual_sol_filtered.groupby("nombre")["year"].count().reset_index().rename(columns={"year":"n_valid_years"}))


Years per station with >= 150 days:


,nombre,n_valid_years
0,BARCELONA AEROPUERTO,46
1,"BARCELONA, FABRA",46
2,MONTSERRAT,21


In [16]:
# --- C: compute_uhi() and regenerate UHI series ---
def compute_uhi(annual_df, urban_station, rural_station="MONTSERRAT"):
    """
    annual_df must contain columns: nombre, year, tmin
    returns merged df with columns year, tmin_urb, tmin_rur, UHI, station
    """
    urb = annual_df[annual_df["nombre"] == urban_station][["year", "tmin"]].rename(columns={"tmin":"tmin_urb"})
    rur = annual_df[annual_df["nombre"] == rural_station][["year", "tmin"]].rename(columns={"tmin":"tmin_rur"})
    merged = pd.merge(urb, rur, on="year", how="inner")
    merged["UHI"] = merged["tmin_urb"] - merged["tmin_rur"]
    merged["station"] = urban_station
    return merged

# Regenerar series UHI (Fabra y BCN Airport)
uhi_fabra = compute_uhi(annual_temp, "BARCELONA, FABRA")
uhi_bcn_airport = compute_uhi(annual_temp, "BARCELONA AEROPUERTO")

# Quick sanity checks
print("Fabra UHI rows:", len(uhi_fabra))
display(uhi_fabra.head())

print("BCN Airport UHI rows:", len(uhi_bcn_airport))
display(uhi_bcn_airport.head())


Fabra UHI rows: 21


,year,tmin_urb,tmin_rur,UHI,station
0,2005,11.532603,9.092329,2.440274,"BARCELONA, FABRA"
1,2006,12.904384,10.509863,2.394521,"BARCELONA, FABRA"
2,2007,12.212329,9.648219,2.564110,"BARCELONA, FABRA"
3,2008,11.718033,9.030874,2.687158,"BARCELONA, FABRA"
4,2009,12.254247,9.040299,3.213948,"BARCELONA, FABRA"


BCN Airport UHI rows: 21


,year,tmin_urb,tmin_rur,UHI,station
0,2005,12.536639,9.092329,3.444310,BARCELONA AEROPUERTO
1,2006,13.338736,10.509863,2.828873,BARCELONA AEROPUERTO
2,2007,12.620548,9.648219,2.972329,BARCELONA AEROPUERTO
3,2008,12.377869,9.030874,3.346995,BARCELONA AEROPUERTO
4,2009,12.653151,9.040299,3.612852,BARCELONA AEROPUERTO


In [17]:
# --- D: Merge UHI with temperature vars (tmin,tmax,tmed,ATD) ---
def merge_uhi_with_temp(uhi_df, annual_df, station_name):
    urb = annual_df[annual_df["nombre"] == station_name][["year","tmin","tmax","tmed","ATD"]]
    merged = pd.merge(uhi_df, urb, on="year", how="inner")
    return merged

uhi_fabra_t = merge_uhi_with_temp(uhi_fabra, annual_temp, "BARCELONA, FABRA")
uhi_bcn_airport_t = merge_uhi_with_temp(uhi_bcn_airport, annual_temp, "BARCELONA AEROPUERTO")

# Show heads
display(uhi_fabra_t.head())
display(uhi_bcn_airport_t.head())


,year,tmin_urb,tmin_rur,UHI,station,tmin,tmax,tmed,ATD
0,2005,11.532603,9.092329,2.440274,"BARCELONA, FABRA",11.532603,19.193425,15.363288,7.660822
1,2006,12.904384,10.509863,2.394521,"BARCELONA, FABRA",12.904384,20.310685,16.610411,7.406301
2,2007,12.212329,9.648219,2.564110,"BARCELONA, FABRA",12.212329,19.763288,15.989041,7.550959
3,2008,11.718033,9.030874,2.687158,"BARCELONA, FABRA",11.718033,19.225137,15.473224,7.507104
4,2009,12.254247,9.040299,3.213948,"BARCELONA, FABRA",12.254247,20.219726,16.237260,7.965479


,year,tmin_urb,tmin_rur,UHI,station,tmin,tmax,tmed,ATD
0,2005,12.536639,9.092329,3.444310,BARCELONA AEROPUERTO,12.536639,20.433333,16.486777,7.896694
1,2006,13.338736,10.509863,2.828873,BARCELONA AEROPUERTO,13.338736,21.462912,17.404396,8.124176
2,2007,12.620548,9.648219,2.972329,BARCELONA AEROPUERTO,12.620548,21.152055,16.884110,8.531507
3,2008,12.377869,9.030874,3.346995,BARCELONA AEROPUERTO,12.377869,20.503552,16.440984,8.125683
4,2009,12.653151,9.040299,3.612852,BARCELONA AEROPUERTO,12.653151,20.862740,16.757808,8.209589


In [18]:
# --- E: Merge with annual_sol_filtered if you want sol present in the *_t dfs ---
# Note: annual_sol_filtered contains only stations with valid sol years (as filtered above)

# For Fabra
if "BARCELONA, FABRA" in annual_sol_filtered["nombre"].unique():
    uhi_fabra_t = pd.merge(uhi_fabra_t, annual_sol_filtered[annual_sol_filtered["nombre"]=="BARCELONA, FABRA"][["year","sol"]], on="year", how="left")

# For BCN Airport
if "BARCELONA AEROPUERTO" in annual_sol_filtered["nombre"].unique():
    uhi_bcn_airport_t = pd.merge(uhi_bcn_airport_t, annual_sol_filtered[annual_sol_filtered["nombre"]=="BARCELONA AEROPUERTO"][["year","sol"]], on="year", how="left")

print("After merging sol (if available):")
display(uhi_fabra_t.head())
display(uhi_bcn_airport_t.head())


After merging sol (if available):


,year,tmin_urb,tmin_rur,UHI,station,tmin,tmax,tmed,ATD,sol
0,2005,11.532603,9.092329,2.440274,"BARCELONA, FABRA",11.532603,19.193425,15.363288,7.660822,7.372603
1,2006,12.904384,10.509863,2.394521,"BARCELONA, FABRA",12.904384,20.310685,16.610411,7.406301,7.360548
2,2007,12.212329,9.648219,2.564110,"BARCELONA, FABRA",12.212329,19.763288,15.989041,7.550959,7.586575
3,2008,11.718033,9.030874,2.687158,"BARCELONA, FABRA",11.718033,19.225137,15.473224,7.507104,6.902459
4,2009,12.254247,9.040299,3.213948,"BARCELONA, FABRA",12.254247,20.219726,16.237260,7.965479,7.428767


,year,tmin_urb,tmin_rur,UHI,station,tmin,tmax,tmed,ATD,sol
0,2005,12.536639,9.092329,3.444310,BARCELONA AEROPUERTO,12.536639,20.433333,16.486777,7.896694,6.973596
1,2006,13.338736,10.509863,2.828873,BARCELONA AEROPUERTO,13.338736,21.462912,17.404396,8.124176,6.527933
2,2007,12.620548,9.648219,2.972329,BARCELONA AEROPUERTO,12.620548,21.152055,16.884110,8.531507,6.414327
3,2008,12.377869,9.030874,3.346995,BARCELONA AEROPUERTO,12.377869,20.503552,16.440984,8.125683,5.915232
4,2009,12.653151,9.040299,3.612852,BARCELONA AEROPUERTO,12.653151,20.862740,16.757808,8.209589,6.377037


In [19]:
# Function to merge UHI df with annual_sol_filtered for a given station
def merge_uhi_with_sol(uhi_df, station_name):
    sol_df = (
        annual_sol_filtered[annual_sol_filtered["nombre"] == station_name][["year","sol"]]
        .drop_duplicates("year")
    )
    return pd.merge(uhi_df, sol_df, on="year", how="inner")


In [33]:
import seaborn as sns
from scipy.stats import pearsonr

# Function to plot UHI vs Insolation scatter with regression line and Pearson r
def scatter_uhi_sol(df, station_label):
    df = df.dropna(subset=["sol", "UHI"])
    x = df["sol"]; y = df["UHI"]
    r, p = pearsonr(x, y)

    plt.figure(figsize=(6,4))
    sns.regplot(x=x, y=y, scatter_kws={"s":45}, line_kws={"color":"orange"})
    plt.title(f"UHI vs Insolation — {station_label}")
    plt.xlabel("Sunshine hours"); plt.ylabel("UHI (°C)")
    plt.grid(True, alpha=0.3)

    plt.gca().text(
    0.05, 0.10, f"r = {r:.2f}\np = {p:.3f}",
    transform=plt.gca().transAxes,
    bbox=dict(facecolor="white", alpha=0.7)
)


    SOL_DIR = os.path.join(os.path.dirname(os.getcwd()), "reports", "sol")
    os.makedirs(SOL_DIR, exist_ok=True)

    filename = f"uhi_vs_sol_{station_label.replace(' ', '_').replace(',', '').lower()}.png"
    save_path = os.path.join(SOL_DIR, filename)

    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()
    


In [34]:
# Merge UHI dfs with sol data
uhi_fabra_s = merge_uhi_with_sol(uhi_fabra, "BARCELONA, FABRA")
uhi_bcn_airport_s = merge_uhi_with_sol(uhi_bcn_airport, "BARCELONA AEROPUERTO")

display(uhi_fabra_s.head())
display(uhi_bcn_airport_s.head())



,year,tmin_urb,tmin_rur,UHI,station,sol
0,2005,11.532603,9.092329,2.440274,"BARCELONA, FABRA",7.372603
1,2006,12.904384,10.509863,2.394521,"BARCELONA, FABRA",7.360548
2,2007,12.212329,9.648219,2.564110,"BARCELONA, FABRA",7.586575
3,2008,11.718033,9.030874,2.687158,"BARCELONA, FABRA",6.902459
4,2009,12.254247,9.040299,3.213948,"BARCELONA, FABRA",7.428767


,year,tmin_urb,tmin_rur,UHI,station,sol
0,2005,12.536639,9.092329,3.444310,BARCELONA AEROPUERTO,6.973596
1,2006,13.338736,10.509863,2.828873,BARCELONA AEROPUERTO,6.527933
2,2007,12.620548,9.648219,2.972329,BARCELONA AEROPUERTO,6.414327
3,2008,12.377869,9.030874,3.346995,BARCELONA AEROPUERTO,5.915232
4,2009,12.653151,9.040299,3.612852,BARCELONA AEROPUERTO,6.377037


In [35]:
# Plot UHI vs Insolation scatter plots
scatter_uhi_sol(uhi_fabra_s, "Fabra (urban-high)")
scatter_uhi_sol(uhi_bcn_airport_s, "BCN Airport")


| Estación               | r         | p         |                   |
| ---------------------- | --------- | --------- | ----------------- |
| **BCN Airport**        | −0.16     | 0.48      | No significativo  |
| **Fabra (urban-high)** | **−0.57** | **0.007** | **Significativo** |


🔍 ¿Qué nos dicen estos números?
🔵 Barcelona Airport

r = −0.16 → correlación muy débil

p = 0.48 → estadísticamente NO significativa

✅ Interpretación
En el aeropuerto la insolación no controla de forma clara la UHI.

Esto es coherente porque:

es un entorno abierto y ventilado

fuerte influencia marítima (brisas, humedad, nubosidad baja)

la energía solar diurna no se traduce directamente en calor nocturno acumulado

👉 Aquí la UHI depende más de:

condiciones sinópticas

viento

estabilidad atmosférica

humedad

No de la insolación anual.

🟠 Fabra (urbano elevado)

r = −0.57 → correlación moderada-fuerte

p = 0.007 → significativa

✅ Interpretación clave (MUY IMPORTANTE)
En Fabra:

A mayor insolación anual → menor UHI

Esto puede parecer contraintuitivo, pero es totalmente correcto en este contexto.

🧠 ¿Por qué la relación es INVERSA en Fabra?

Aquí está la clave climatológica 👇

En Fabra:

más insolación → más cielos despejados

cielos despejados → mejor enfriamiento nocturno

mejor enfriamiento → Tmin más baja

Tmin baja → menor diferencia urbano–rural

⇒ UHI más débil

Es decir:

El efecto del enfriamiento radiativo nocturno domina sobre el calentamiento diurno.

Esto es típico de:

estaciones elevadas

climas mediterráneos

situaciones anticiclónicas secas

Por eso:

años nublados → noches más “tapadas” → menos enfriamiento → UHI más fuerte

años muy soleados → noches claras → mucho enfriamiento → UHI más débil

✅ Resultado físicamente coherente y muy interesante.

🔥 Comparación clave entre estaciones (mensaje fuerte del TFG)

BCN Airport:
Insolación ≠ controlador de UHI → domina el régimen costero.

Fabra:
Insolación SÍ modula la UHI, pero a través del enfriamiento nocturno, no del calentamiento diurno.

👉 Esto demuestra que:

El mismo factor climático (sol) puede tener efectos opuestos según el contexto urbano, topográfico y atmosférico.

Esto es un argumento de nivel alto en climatología urbana.

The correlation between annual sunshine duration and UHI intensity reveals contrasting behaviours between stations.
At Barcelona Airport, no statistically significant relationship is observed, indicating that solar radiation plays a minor role under strong maritime influence.
In contrast, Fabra shows a significant negative correlation (r = −0.57, p < 0.01), suggesting that higher insolation years enhance nocturnal radiative cooling under clear-sky conditions, resulting in lower Tmin and weaker UHI.
This highlights the dominant role of nighttime cooling processes over daytime heating in elevated urban environments.